(1) Import required packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

(2) Provide the path here at which find input data and save the output

In [ ]:
direc = '/Users/snagel/Dropbox/hdrive/teaching/Booth/AdvancedInvestments/2024/output/'

(3) Set parameters 

In [ ]:
burn = 120  # (default = 120) length of burn-in period
oos = True  #  out-of-sample or in-sample
stocks = False  # stocks or bonds

(4) Read data 

In [ ]:
M = pd.read_excel('PS3data1.xlsx')  
ret = M[['rvwind', 'rtreas']].values
rf = M['rf'].values

if stocks:
    fn1 = 'stocks'
    retm = ret[:, 0]
else:
    fn1 = 'bonds'
    retm = ret[:, 1]

fullretm = retm
fullrf = rf
retm = retm[burn:]
rf = rf[burn:]

T = len(fullretm)

(5) Calculate log returns and full-sample variance 

In [ ]:
logrf = np.log(1 + rf)
logret = np.log(1 + ret)
logretm = np.log(1 + retm)
logfullretm = np.log(1 + fullretm)
logfullrf = np.log(1 + fullrf)
v = np.var(logfullretm - logfullrf,ddof=1) 

(6) Calculate portfolio weights 

In [ ]:
gamma = np.arange(0.5, 1.6, 0.25)
G = len(gamma)
if oos:
    fn2 = 'oos'
    t = np.arange(1, T + 1)
    m = np.log(np.cumsum(np.exp(logfullretm - logfullrf)) / t)
    m = m[burn-1:-1]
    w = np.tile(m.reshape(-1, 1), (1, G)) / np.tile(gamma*v, (T-burn, 1))
else:
    fn2 = 'is'
    m = np.log(np.mean(np.exp(logfullretm - logfullrf)))
    w = np.tile(m / (gamma * v), (T, 1))
    logretm = logfullretm
    logrf = logfullrf
    retm = fullretm
    rf = fullrf

(7) Portfolio returns based on continuous rebalancing approximation

In [ ]:
K = w.shape[1]
logexc = logretm - logrf
logrp = np.tile(logrf.reshape(-1,1), (1, K)) + np.tile(logexc.reshape(-1,1), (1, K)) * w + v * w * (1 - w) / 2
cumlogrp = np.cumsum(logrp, axis=0)
cumrp = np.exp(cumlogrp)

(8) Plot weights

In [ ]:
plt.figure()
plt.plot(w)
plt.legend([f'$\gamma = {g}$' for g in gamma], loc='upper left', fontsize=10)
plt.xlabel('Time')
plt.ylabel('Weight')
plt.grid(True)
plt.savefig(direc + fn1 + fn2 + 'weights.pdf', format='pdf')
plt.show()

(9) Plot log wealth outcomes 

In [ ]:
f = plt.figure()
plt.plot(cumlogrp, linewidth=2)
plt.legend([f'$\gamma = {g}$' for g in gamma], loc='upper left', fontsize=10)
plt.xlabel('Time')
plt.ylabel('Log Wealth')
plt.grid(True)
plt.savefig(direc + fn1 + fn2 + 'wealth.pdf', format='pdf')
plt.show()

(10) Monthly rebalancing

In [ ]:
rpx = np.tile(retm.reshape(-1, 1), (1, K)) * w + np.tile(rf.reshape(-1, 1), (1, K)) * (1 - w)
cumrpx = np.cumprod(1 + rpx, axis=0)
cumlogw = np.log(cumrpx)

f = plt.figure()
plt.plot(cumlogw, linewidth=2)
plt.legend([f'$\gamma = {g}$' for g in gamma], loc='upper left', fontsize=10)
plt.xlabel('Time')
plt.ylabel('Log Wealth')
plt.grid(True)
plt.show()